## 12차시: [실습] Open DART API로 기업 공시정보 자동 수집

### 학습 목표
- Open DART API를 활용하여 특정 기업의 공시 목록을 파이썬으로 자동 수집
- 공시 데이터 필터링·분류(정기/수시/주요), 정제 및 분석
- 재무제표 데이터(OpenDART fnlttSinglAcnt) 조회 및 핵심 항목 추출
- 공시 원문 URL 연계(클릭 가능한 링크/HTML 표시) 및 종합 자동화 함수 작성

### 학습 내용
1. API 키 설정
2. 기업 고유번호 기반 공시 목록 조회 (최근 3개월 등 기간 설정)
3. 기간·유형별 공시 검색 및 공시유형(A~E) 집계
4. 재무제표 데이터 조회 및 주요 계정  추출
5. 공시 원문 URL 생성 및 링크 표시
6. 공시 분류(정기/수시/주요) + 데이터 정제 + 월별 건수 시각화
7. 종합 자동화 함수  


### 서비스 엔드 포인트
| 서비스(Endpoint)             | 설명              |
| --------------------- | --------------- |
| `company.json`        | 기업개황 조회         |
| `fnlttSinglAcnt.json` | 단일회사 전체 재무제표    |
| `fnlttMultiAcnt.json` | 다중회사 주요 계정 재무정보 |
| `list.json`           | 공시 목록 조회        |
| `document.xml`        | 공시 원문 조회        |

In [1]:
!pip install -q koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import requests
import time
import os
from dotenv import load_dotenv

---
## 1. API 키 설정

In [3]:
# API 키 로드
from google.colab import files

print("[.env 파일 업로드]")
print("=" * 60)
print("로컬에 저장된 .env 파일을 선택해주세요.")
print("(DART_API_KEY가 포함된 파일)")
print()

uploaded = files.upload()

# .env 파일 로드
load_dotenv('.env')

DART_API_KEY = os.getenv('DART_API_KEY')

# API 키 로드 확인
print("\n[API 키 로드 상태]")
print("=" * 60)
if DART_API_KEY:
    print(f"DART API Key: 설정완료 ({DART_API_KEY[:8]}...)")
else:
    print("DART API Key: 미설정")

[.env 파일 업로드]
로컬에 저장된 .env 파일을 선택해주세요.
(DART_API_KEY가 포함된 파일)



Saving .env to .env

[API 키 로드 상태]
DART API Key: 설정완료 (1493d0f2...)


---
## 2. 기업 고유번호 기반 공시 조회

기업 고유번호(corp_code)를 이용하여 공시 정보를 조회합니다.

In [4]:
# 기업 고유번호 (예시: 삼성전자)
corp_code = "00126380"
corp_name = "삼성전자"

print("[분석 대상 기업]")
print("=" * 60)
print(f"  - {corp_name}: {corp_code}")

[분석 대상 기업]
  - 삼성전자: 00126380


In [5]:
# 기업 고유번호 기반 공시 조회 함수
def get_disclosures_by_corp_code(corp_code, start_date=None, end_date=None,
                                  page_count=100, pblntf_ty=None, report_nm_keyword=None):
    """
    특정 기업의 공시 목록을 조회하는 함수

    Parameters:
    -----------
    corp_code : 기업 고유번호
    start_date : 시작일 (YYYYMMDD, None이면 최근 1년)
    end_date : 종료일 (YYYYMMDD, None이면 오늘)
    page_count : 페이지당 조회 건수 (기본값: 100)
    pblntf_ty : 공시유형 (A:정기공시, B:주요사항보고서, C:발행공시, D:지분공시, E:기타공시, None:전체)
    report_nm_keyword : 보고서명 키워드 (예: "사업보고서", "반기보고서")

    Returns:
    --------
    DataFrame : 공시 정보 DataFrame
    """
    # 기본값 설정
    if end_date is None:
        end_date = datetime.now().strftime('%Y%m%d')
    if start_date is None:
        start_date = (datetime.now() - timedelta(days=365)).strftime('%Y%m%d')

    # OpenDART 공시 목록 조회 API 엔드포인트
    url = "https://opendart.fss.or.kr/api/list.json"

    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bgn_de": start_date,
        "end_de": end_date,
        "page_count": str(page_count)
    }

    if pblntf_ty:   # 공시 유형이 지정된 경우
        params["pblntf_ty"] = pblntf_ty

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:   # HTTP 요청이 정상적으로 처리
            data = response.json()

            if data['status'] == '000':    # OpenDART API 정상 응답
                disclosures = data['list']      # 공시 목록(list)을 추출
                df = pd.DataFrame(disclosures)    # 공시 목록을 DataFrame으로 변환

                # 보고서명 키워드 필터링
                if not df.empty and report_nm_keyword:
                    df = df[df['report_nm'].str.contains(report_nm_keyword, na=False, regex=False)].copy()

                return df
            else:
                print(f"오류: {data.get('message', '')}")
                return pd.DataFrame()
        else:
            print(f"HTTP 오류: {response.status_code}")
            return pd.DataFrame()

    except Exception as e:
        print(f"예외 발생: {e}")
        return pd.DataFrame()

In [6]:
# 최근 3개월 공시 정보 수집
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

start_str = start_date.strftime('%Y%m%d')
end_str = end_date.strftime('%Y%m%d')

print(f"[{corp_name} 최근 3개월 공시 정보 수집]")
print(f"기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
print()

df_disclosures = get_disclosures_by_corp_code(corp_code, start_str, end_str, page_count=100)

if not df_disclosures.empty:
    df_disclosures['corp_name'] = corp_name
    print(f"총 {len(df_disclosures)}건의 공시 정보 수집 완료")
else:
    print("수집된 공시 정보가 없습니다.")

df_disclosures.head()

[삼성전자 최근 3개월 공시 정보 수집]
기간: 2025-10-11 ~ 2026-01-09

총 100건의 공시 정보 수집 완료


,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,최대주주등소유주식변동신고서,20260108800667,삼성전자,20260108,유
1,00126380,삼성전자,005930,Y,기업설명회(IR)개최(안내공시),20260108800002,삼성전자,20260108,유
2,00126380,삼성전자,005930,Y,연결재무제표기준영업(잠정)실적(공정공시),20260108800001,삼성전자,20260108,유
3,00126380,삼성전자,005930,Y,주요사항보고서(자기주식취득결정),20260107000715,삼성전자,20260107,
4,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20251231000603,홍창표,20251231,


In [7]:
# 주요 컬럼만 선택하여 보기 좋게 정리
display_cols = ['rcept_dt', 'corp_name', 'report_nm', 'flr_nm', 'rcept_no']
available_cols = [col for col in display_cols if col in df_disclosures.columns]

df_display = df_disclosures[available_cols].copy()
df_display.columns = ['접수일자', '기업명', '보고서명', '제출인', '접수번호']
df_display.head()

,접수일자,기업명,보고서명,제출인,접수번호
0,20260108,삼성전자,최대주주등소유주식변동신고서,삼성전자,20260108800667
1,20260108,삼성전자,기업설명회(IR)개최(안내공시),삼성전자,20260108800002
2,20260108,삼성전자,연결재무제표기준영업(잠정)실적(공정공시),삼성전자,20260108800001
3,20260107,삼성전자,주요사항보고서(자기주식취득결정),삼성전자,20260107000715
4,20251231,삼성전자,임원ㆍ주요주주특정증권등소유상황보고서,홍창표,20251231000603


---
## 3. 공시 원문 URL 연계

공시 원문을 조회할 수 있는 URL을 생성합니다.

In [8]:
from IPython.display import display, HTML

# 공시 원문 URL 생성 함수
def generate_document_url(rcept_no):
    """공시 원문을 조회할 수 있는 URL을 생성하는 함수"""
    return f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcept_no}"

# 공시 원문 URL 추가
df_disclosures['공시원문URL'] = df_disclosures['rcept_no'].apply(generate_document_url)

url_display_cols = ['rcept_dt', 'corp_name', 'report_nm', 'rcept_no', '공시원문URL']

# (1) 링크가 클릭되도록 HTML로 렌더링
df_show = df_disclosures[url_display_cols].head(5).copy()
df_show['공시원문URL'] = df_show['공시원문URL'].apply(lambda u: f'<a href="{u}" target="_blank">{u}</a>')

display(HTML(df_show.to_html(index=False, escape=False)))

rcept_dt,corp_name,report_nm,rcept_no,공시원문URL
20260108,삼성전자,최대주주등소유주식변동신고서,20260108800667,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20260108800667
20260108,삼성전자,기업설명회(IR)개최(안내공시),20260108800002,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20260108800002
20260108,삼성전자,연결재무제표기준영업(잠정)실적(공정공시),20260108800001,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20260108800001
20260107,삼성전자,주요사항보고서(자기주식취득결정),20260107000715,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20260107000715
20251231,삼성전자,임원ㆍ주요주주특정증권등소유상황보고서,20251231000603,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20251231000603


---
## 4. 기간·유형별 공시 검색

기간, 공시 유형을 조합하여 공시를 검색합니다.

In [9]:
print("\n[공시유형별 집계]")
print("=" * 60)

# 공시 유형 코드 → 한글명 매핑
PBLNTF_TYPE_MAP = {
    'A': '정기공시',
    'B': '주요사항보고서',
    'C': '발행공시',
    'D': '지분공시',
    'E': '기타공시'
}

summary = []  # 집계 결과 저장용 리스트

for pblntf_code, pblntf_name in PBLNTF_TYPE_MAP.items():
    df = get_disclosures_by_corp_code(
        corp_code=corp_code,
        start_date=start_str,
        end_date=end_str,
        pblntf_ty=pblntf_code
    )

    count = len(df) if not df.empty else 0

    summary.append({
        "공시유형코드": pblntf_code,
        "공시유형": pblntf_name,
        "공시건수": count
    })

    print(f"{pblntf_name} ({pblntf_code}) : {count}건")

df_summary = pd.DataFrame(summary)

df_summary


[공시유형별 집계]
정기공시 (A) : 1건
주요사항보고서 (B) : 2건
오류: 조회된 데이타가 없습니다.
발행공시 (C) : 0건
지분공시 (D) : 93건
기타공시 (E) : 1건


,공시유형코드,공시유형,공시건수
0,A,정기공시,1
1,B,주요사항보고서,2
2,C,발행공시,0
3,D,지분공시,93
4,E,기타공시,1


---
## 5. 재무제표 데이터 제공

Open DART API를 통해 재무제표 데이터를 조회합니다.

In [10]:
# 재무제표 조회 함수
def get_financial_statements(corp_code, bsns_year, reprt_code, fs_div='CFS'):
    """
    단일 회사의 재무제표를 조회하는 함수

    Parameters:
    -----------
    corp_code : 기업 고유번호 (8자리, 종목코드 아님)
    bsns_year : 사업연도 (YYYY)
    reprt_code : 보고서 코드 (11013:1분기, 11012:반기, 11014:3분기, 11011:사업보고서)
    fs_div : 재무제표 구분 (CFS:연결, OFS:별도)

    Returns:
    --------
    DataFrame : 재무제표 데이터
    """
    # 단일회사 전체 재무제표 엔드포인트
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"

    params = {
        "crtfc_key": DART_API_KEY,      # OpenDART 인증키
        "corp_code": corp_code,         # 기업 고유번호 (8자리)
        "bsns_year": str(bsns_year),    # 사업연도 (문자열로 변환)
        "reprt_code": str(reprt_code),  # 보고서 코드
        "fs_div": fs_div                    # 재무제표 구분 (CFS / OFS)
    }

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

            if data['status'] == '000':
                df = pd.DataFrame(data['list'])
                return df
            else:
                print(f"오류: {data.get('message', '')}")
                return pd.DataFrame()
        else:
            print(f"HTTP 오류: {response.status_code}")
            return pd.DataFrame()

    except Exception as e:
        print(f"예외 발생: {e}")
        return pd.DataFrame()

In [11]:
# 최근 사업보고서 재무제표 조회
financial_year = "2024"
print(f"\n[{corp_name} {financial_year}년 사업보고서 주요 재무제표 조회]")
print("=" * 60)

# 보고서 코드: 11011 (사업보고서)
df_financial = get_financial_statements(
    corp_code=corp_code,
    bsns_year=financial_year,
    reprt_code=11011,
    fs_div='CFS'
)

account_nm_list = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
df_financial_filtered = df_financial[df_financial['account_nm'].isin(account_nm_list)].copy()

display_cols = ['account_nm', 'thstrm_amount', 'frmtrm_amount']
if all(col in df_financial_filtered.columns for col in display_cols):
    df_financial_display = df_financial_filtered[display_cols].copy()
    df_financial_display.columns = ['계정명', '당기금액', '전기금액']

df_financial_display


[삼성전자 2024년 사업보고서 주요 재무제표 조회]


,계정명,당기금액,전기금액
2,자산총계,"514,531,948,000,000","455,905,980,000,000"
5,부채총계,"112,339,878,000,000","92,228,115,000,000"
8,자본총계,"402,192,070,000,000","363,677,865,000,000"
9,매출액,"300,870,903,000,000","258,935,494,000,000"
10,영업이익,"32,725,961,000,000","6,566,976,000,000"
16,자산총계,"324,966,127,000,000","296,857,289,000,000"
19,부채총계,"88,569,470,000,000","72,069,515,000,000"
22,자본총계,"236,396,657,000,000","224,787,774,000,000"
23,매출액,"209,052,241,000,000","170,374,090,000,000"
24,영업이익,"12,361,034,000,000","-11,526,297,000,000"


---
## 6. 데이터 분석 및 자동화 활용

위에서 배운 함수들을 연결하여 관심 공시를 자동으로 체크하는 함수를 작성합니다.

In [17]:
# 관심 공시 자동 체크 함수
def check_disclosures(corp_code, corp_name, keyword=None, pblntf_ty=None, start_date=None, end_date=None):
    """
    관심있는 공시 내용이 나왔는지 자동으로 체크하는 함수

    Parameters:
    -----------
    corp_code : 기업 고유번호
    corp_name : 기업명
    keyword : 관심 키워드 (예: "배당", "임원", "사업보고서", None이면 전체)
    pblntf_ty : 공시유형 (A:정기공시, B:주요사항보고서, C:발행공시, D:지분공시, E:기타공시, None:전체)
    start_date : 시작일 (YYYYMMDD, None이면 최근 3개월)
    end_date : 종료일 (YYYYMMDD, None이면 오늘)

    Returns:
    --------
    DataFrame : 조건에 맞는 공시 정보 DataFrame
    """
    print(f"[{corp_name} 관심 공시 체크]")
    print("=" * 60)
    print(f"기간: {start_date} ~ {end_date}")

    if keyword:
        print(f"키워드: {keyword}")
    if pblntf_ty:
        pblntf_ty_names = {'A': '정기공시', 'B': '주요사항보고서', 'C': '발행공시',
                          'D': '지분공시', 'E': '기타공시'}
        print(f"공시유형: {pblntf_ty_names.get(pblntf_ty, pblntf_ty)}")
    print()

    # 1. 공시 정보 수집 (위에서 배운 함수 사용)
    df = get_disclosures_by_corp_code(corp_code, start_str, end_str, pblntf_ty=pblntf_ty, report_nm_keyword=keyword)

    if df.empty:
        print("✗ 조건에 맞는 공시가 없습니다.")
        return pd.DataFrame()

    df['corp_name'] = corp_name

    # 2. 공시 원문 URL 추가 (위에서 배운 함수 사용)
    if 'rcept_no' in df.columns:
        df['공시원문URL'] = df['rcept_no'].apply(generate_document_url)

    # 결과 출력
    print(f"✓ {len(df)}건 발견!")
    print("\n[발견된 공시 목록]")
    print("=" * 60)

    display_cols = ['rcept_dt', 'corp_name', 'report_nm', 'rcept_no', '공시원문URL']
    available_cols = [col for col in display_cols if col in df.columns]
    df_display = df[available_cols].copy()
    df_display.columns = ['접수일자', '기업명', '보고서명', '접수번호', '공시원문URL']

    print(df_display.to_string(index=False))

    return df

In [18]:
# 관심 공시 체크 예시
start_date = "20240101"
end_data="20241231"

# 예시 1: 배당 관련 공시 체크
print("[예시 1] 배당 관련 공시 체크")
print("=" * 60)
df_dividend = check_disclosures(corp_code, corp_name, keyword="배당",
                                start_date=start_date, end_date=end_date)
print()

# 예시 2: 사업보고서 체크
print("[예시 2] 사업보고서 체크")
print("=" * 60)
df_annual = check_disclosures(corp_code, corp_name, keyword="사업보고서",
                              start_date=start_date, end_date=end_date)
print()

# 예시 3: 정기공시만 체크
print("[예시 3] 정기공시 체크")
print("=" * 60)
df_regular = check_disclosures(corp_code, corp_name, pblntf_ty='A',
                               start_date=start_date, end_date=end_date)

[예시 1] 배당 관련 공시 체크
[삼성전자 관심 공시 체크]
기간: 20240101 ~ 2026-01-09 02:09:26.448684
키워드: 배당

✗ 조건에 맞는 공시가 없습니다.

[예시 2] 사업보고서 체크
[삼성전자 관심 공시 체크]
기간: 20240101 ~ 2026-01-09 02:09:26.448684
키워드: 사업보고서

✗ 조건에 맞는 공시가 없습니다.

[예시 3] 정기공시 체크
[삼성전자 관심 공시 체크]
기간: 20240101 ~ 2026-01-09 02:09:26.448684
공시유형: 정기공시

✓ 1건 발견!

[발견된 공시 목록]
    접수일자  기업명            보고서명           접수번호                                                     공시원문URL
20251114 삼성전자 분기보고서 (2025.09) 20251114002447 https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20251114002447


---
## 학습 정리

### 1. 기업 고유번호 기반 공시 조회
- `get_disclosures_by_corp_code()`: 단일 기업 공시 조회
- API 호출 제한 고려 (`time.sleep()`)

### 2. 기간·유형별 공시 검색
- `get_disclosures_by_corp_code()`: 다양한 조건으로 공시 검색
- 공시유형 필터링 (A:정기, B:주요, C:발행, D:지분, E:기타)
- 보고서명 키워드 검색

### 3. 정기·수시·주요 공시 구분
- `classify_disclosure_type()`: 공시 분류 함수
- 보고서명과 공시유형 코드 기반 자동 분류
- 정기공시, 수시공시, 주요공시, 기타로 구분

### 4. 재무제표 데이터 제공
- `get_financial_statements()`: 단일 회사 재무제표 조회
- 연결/별도 재무제표 선택 가능

### 5. 공시 원문 URL 연계
- `generate_document_url()`: 공시 원문 URL 생성
- DART 공시 시스템과 연계

### 6. 데이터 분석 및 자동화
- `check_disclosures()`: 관심 공시 자동 체크 함수
- 위에서 배운 함수들(`get_disclosures_by_corp_code`, `generate_document_url`)을 연결하여 사용
- 키워드나 공시유형으로 원하는 공시만 빠르게 확인

---

### 다음 차시 예고
- 13차시: [실습] FRED API로 경제지표 수집
  - 미국 경제지표 자동 수집
  - 여러 지표 비교 분석